# Practice 6 Part 2

1. Importing libraries
2. Load and prepare samsum dataset
3. Fine-tune and evaluate FLAN-T5 with LoRA 
4. Run Inference and Experiment with Prompts
5. Conclusions about metric


This notebook is a twin of a last one, but here we are using LoRA to fine-tune our model. At the and of notebook there will be a conclusion about metrics we used to evaluate our model.

## 1. Importing libraries

In [1]:

!pip install pytesseract accelerate bitsandbytes transformers datasets bleu sacrebleu rouge-score nltk tensorboard py7zr --upgrade
!pip install peft
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 12.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 79.6 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 37.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 14.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 67.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 85.1 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 92.5 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 71.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.3/412.3 kB 35.5 MB/s eta 0:00

In [2]:
import torch

# Check for CUDA availability for GPU training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'device {device}')

device cuda


## 2. Load and prepare samsum dataset


In [3]:
dataset_id = "samsum"

In [4]:
from datasets import load_dataset

# Load dataset from the hub
dataset = load_dataset(dataset_id)

dataset

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [5]:
print(f"Train dataset size: {len(dataset['train'])}")
print(f"Test dataset size: {len(dataset['test'])}")

Train dataset size: 14732
Test dataset size: 819


In [6]:
from random import randrange        


sample = dataset['train'][randrange(len(dataset["train"]))]
print(f"dialogue: \n{sample['dialogue']}\n---------------")
print(f"summary: \n{sample['summary']}\n---------------")

dialogue: 
Ella: Could you buy some bread?
Nathan: multigrain or wholewheat?
Ella: never wholewheat!
Nathan: hahah, ok!
---------------
summary: 
Nathan will buy some multigrain bread.
---------------


In [7]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_id="google/flan-t5-small"

# Load tokenizer of FLAN-t5-small
tokenizer = AutoTokenizer.from_pretrained(model_id)


In [8]:
from datasets import concatenate_datasets

# The maximum total input sequence length after tokenization. 
# Sequences longer than this will be truncated, sequences shorter will be padded.
tokenized_inputs = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["dialogue"], truncation=True), batched=True, remove_columns=["dialogue", "summary"])
max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
print(f"Max source length: {max_source_length}")

# The maximum total sequence length for target text after tokenization. 
# Sequences longer than this will be truncated, sequences shorter will be padded.
tokenized_targets = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["summary"], truncation=True), batched=True, remove_columns=["dialogue", "summary"])
max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])
print(f"Max target length: {max_target_length}")

Map:   0%|          | 0/15551 [00:00<?, ? examples/s]

Max source length: 512


Map:   0%|          | 0/15551 [00:00<?, ? examples/s]

Max target length: 95


In [9]:
def preprocess_function(sample,padding="max_length"):
    # add prefix to the input for t5
    inputs = ["summarize: " + item for item in sample["dialogue"]]

    # tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample["summary"], max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=["dialogue", "summary", "id"])
print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

Keys of tokenized dataset: ['input_ids', 'attention_mask', 'labels']


## 3. Fine-tune and evaluate FLAN-T5

After we have processed our dataset, we can start training our model. Therefore we first need to load our FLAN-T5 from the Hugging Face. 

In [16]:
from transformers import AutoModelForSeq2SeqLM

model_id="google/flan-t5-small"


model = AutoModelForSeq2SeqLM.from_pretrained(model_id, load_in_8bit=True)

In [17]:
# Now, we can prepare our model for the LoRA int-8 training using peft.

from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training, TaskType

# Define LoRA Config 
lora_config = LoraConfig(
 r=16, 
 lora_alpha=32,
 target_modules=["q", "v"],
 lora_dropout=0.05,
 bias="none",
 task_type=TaskType.SEQ_2_SEQ_LM
)
# prepare int-8 model for training
model = prepare_model_for_int8_training(model)

# add LoRA adaptor
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 688,128 || all params: 77,649,280 || trainable%: 0.8862001038515747


/opt/conda/lib/python3.10/site-packages/peft/utils/other.py:135: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


In [18]:
import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
from datasets import load_metric


nltk.download("punkt")

# Metric
metric_rouge = evaluate.load("rouge")
metric_bleu = load_metric("sacrebleu")

# Helper function to postprocess text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    # Compute ROUGE scores
    rouge_scores = metric_rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    rouge_scores = {k: round(v * 100, 4) for k, v in rouge_scores.items()}
    
    # Compute BLEU score
    references = [[label.split()] for label in decoded_labels]
    predictions = [pred.split() for pred in decoded_preds]
    bleu_score = metric_bleu.compute(predictions=predictions, references=references)
    bleu_score = {'bleu': round(bleu_score['score'], 4)}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result = {**rouge_scores, **bleu_score, "gen_len": round(np.mean(prediction_lens), 4)}
    return result

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [19]:
from transformers import DataCollatorForSeq2Seq

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)


In [20]:
from huggingface_hub import HfFolder
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments


output_id = f"{model_id.split('/')[1]}-{dataset_id}"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=output_id,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    fp16=False, # Overflows with fp16
		auto_find_batch_size=True,
    learning_rate=1e-3, # higher learning rate
    num_train_epochs=5, # number of epochs we want our model to train
    # logging & evaluation strategies
    logging_dir=f"{output_id}/logs",
    logging_strategy="steps",
    logging_steps=500,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    # metric_for_best_model="overall_f1"
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)

In [ ]:
# Start training
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Bleu,Gen Len
1,1.864900,1.692618,42.123200,18.424800,35.106200,38.509200,20.387000,16.715500


/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


In [ ]:
trainer.evaluate()

## 4. Run Inference


In [ ]:
from transformers import pipeline
from random import randrange        

sample = dataset['test'][randrange(len(dataset["test"]))]

prompt = "Summarize the following dialogue. " + sample['dialogue']

input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to('cuda')  # Ensure input is on the same device as the model

# Generate the summary
summary_ids = model.generate(input_ids, max_length=100, num_beams=4, length_penalty=2.0, early_stopping=True)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print(f"dialogue: \n{sample['dialogue']}\n")
print("-------------------------------------------")
print(f"Summary: {sample['summary']}\n")
print("-------------------------------------------")
print(f"Model's Summary: {summary}\n")

In [ ]:
# Experiment with Prompts
prompts = [
    "Summarize the following dialogue.",
    "Provide a concise summary of the conversation.",
    "Answer the question in a brief summary.",
    "Capture the emotional tone of the conversation.",
    "Provide a summary that captures the context and key points.",
    "Summarize the dialogue with a focus on opinions and preferences.",
    "Summarize the roles and responsibilities discussed in the conversation.",
    "Compare and contrast the views in the conversation.",
]
    
for prompt in prompts:
    print(f"Prompt: {prompt}")
    input_text = f"{prompt} NATASHA: What time is it? We’re going to be late! TONY: It’s a quarter after seven. We’re on time. Don’t panic. NATASHA: But I thought we had to be at the restaurant by 7:30 for the surprise party. We’ll never make it there with all this evening traffic. TONY: Sure we will. Rush hour is almost over. Anyway, the party starts at 8:00. But I do need help with directions. Can you call the restaurant and ask them where we park our car?"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')  # Ensure input is on the same device as the model

    # Generate the summary
    summary_ids = model.generate(input_ids, max_length=150, num_beams=4, length_penalty=2.0, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    print(f"Summary: {summary}\n")

## 5. Conclusions about metric


We've used BLUE and ROUGE scores to evaluate our model for summarization task, so let's describe their pros and cons in general:

BLEU:

* Pros:

 * BLEU is relatively easy to compute and understand.
 * BLEU is good at measuring the fluency and grammatical correctness of generated text.
 
* Cons:

 * BLEU is less good at measuring the informativeness and coherence of generated text.
 * BLEU can be sensitive to paraphrasing and word order changes.
 * BLEU can be biased towards longer generations.

ROUGE:

* Pros:

 * ROUGE is better at measuring the informativeness and coherence of generated text than BLEU.
 * ROUGE is less sensitive to paraphrasing and word order changes than BLEU.
 * ROUGE is less biased towards longer generations than BLEU.

* Cons:

 * ROUGE is more computationally expensive to compute than BLEU.
 * ROUGE is more difficult to understand than BLEU.
 * ROUGE can be less reliable for short generations.

Relative to the generations of a summarization model:

Both BLEU and ROUGE can be used to evaluate the quality of the summaries generated by a summarization model. However, ROUGE is generally considered to be a better metric for summarization than BLEU. This is because ROUGE better captures the informativeness and coherence of summaries.

---

Overall, both BLEU and ROUGE can be useful metrics for evaluating the quality of the summaries generated by a summarization model. However, it is important to understand the strengths and weaknesses of each metric and to use a combination of metrics to get a more complete picture of the quality of the generations.